## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
i = 0
draw_img = None
# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    draw_img = np.copy(img)
    
    # Convert to GrayScale image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(draw_img, (9,6), corners, ret)
# Use cv2.calibrateCamera to get distortion coefficients and calibration matrix        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (draw_img.shape[0], draw_img.shape[1]), None, None)


## Undistort Chessboard Images

In [3]:
# initializing counter to save image as Undistort1.jpg, Undistort2.jpg etc...
i = 0
# Create list of images to undistort
images = ['camera_cal/calibration1.jpg','camera_cal/calibration2.jpg','camera_cal/calibration3.jpg',
          'camera_cal/calibration4.jpg','camera_cal/calibration5.jpg']
# Looping through above images to undistort and save
for fname in images:
    # Read image
    img = cv2.imread(fname)
    draw_img = np.copy(img)
    # Increment counter
    i += 1
    # cv2.undistort()
    dest = cv2.undistort(img, mtx, dist, None, mtx)
    # Define path to save the figure
    path = 'output_images/Undistorted_Images/Undistort'+str(i)+'.jpg'
    # Define figure
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    f.tight_layout()
    # Add Original Image to figure
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=25)
    # Add Undistorted image to figure
    ax2.imshow(dest)
    ax2.set_title('Undistorted Image', fontsize=25)
    # Save figure
    plt.savefig(path)

## Calculate tranformation matrix and calibration matix using Image & Object points

In [4]:
# Define source 'src' and destination 'dst' points
# I have hardcoded this values for this project, can be generic using formula in writeup.md file
src = np.float32([[585, 460], [203, 720], [1127, 720], [695, 460]])
dst = np.float32([[320, 0], [320, 720], [960, 720], [960, 0]])
# Compute tranform matrix M using cv2.getPerspectiveTransform()
M = cv2.getPerspectiveTransform(src, dst)
# Compute inverse tranform matrix Minv using cv2.getPerspectiveTransform()
Minv = cv2.getPerspectiveTransform(dst, src)

## Find binary warped image using color conversion, gradient etc..

In [5]:
def get_warped_images(image, s_thresh=(100, 255), sx_thresh=(20, 100), caller = False):
    #Use cv2.undistort() to undistort image
    dest = cv2.undistort(image, mtx, dist, None, mtx)
    img = np.copy(dest)
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    # Extracting 'L' channel from image 'hls'
    l_channel = hls[:,:,1]
    # Extracting 'S' channel from image 'hls'
    s_channel = hls[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Define combined and final image 
    combined = np.zeros_like(s_binary)
    final = np.zeros_like(s_binary)
    # Combining image 'sxbinary' and 's_binary' using OR operator 
    combined[(sxbinary == 1) | (s_binary == 1)] = 1
    
    # Experiment done to make broken lane line continuous using hough transform
    '''combined = combined.astype(np.uint8)
    final = combined
    #color_binary = np.dstack(( np.zeros_like(s_binary), sxbinary, s_binary))
    
    mask = np.zeros_like(combined)
    ignore_mask_color = 255
    
    # This time we are defining a four sided polygon to mask
    vertices = np.array([[(800, 450), (1150, 680), (150, 680), (500, 450)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    combined1 = cv2.bitwise_and(combined, mask)
    
    #vertices = np.array([[(580, 450), (770, 450), (1150, 680), (350, 680)]], dtype=np.int32)
    #masked_edges = region_of_interest(combined, vertices)
    line_image = hough_lines(combined1, 2, np.pi/180, 40, 60, 100)
    line_image = line_image[:,:,0]
    result = weighted_img(line_image, combined)
    
    combined = result // 255'''
    
    # Define mask image to mask left and right lane lines.
    mask = np.zeros_like(combined)
    ignore_mask_color = 1
    
    # This time we are defining a four sided polygon to mask left lane
    vertices = np.array([[(580, 460), (620, 460), (350, 680), (150, 680)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    combined1 = cv2.bitwise_and(combined, mask)
    
    # This time we are defining a four sided polygon to mask right lane
    vertices = np.array([[(710,460),(760, 460), (1200, 680), (1000,680)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    combined2 = cv2.bitwise_and(combined, mask)
    
    # Combining mask images 'combined1' and 'combined2' using OR operator
    final[(combined1 == 1) | (combined2 == 1)] = 1
    combined = final
    
    # Getting warped image using perspective transform
    binary_warped, warped = get_perspective_transform(combined, image)
    
    # Return all images if call is from test images pipeline
    # else return only 'binary_warped' and 'dest' images, if call is from pipeline for videos
    if caller:
        return binary_warped, warped, dest, combined
    else:
        return binary_warped, dest

In [6]:
def get_perspective_transform(combined, image):
    combined_size = combined.shape[::-1]
    # use cv2.warpPerspective to get binary warped image 'binary_warped' using binary image 'combined'
    binary_warped = cv2.warpPerspective(combined, M, combined_size, flags = cv2.INTER_LINEAR)
    # use cv2.warpPerspective to get warped image 'binary_warped' using 3 channel image 'image'
    warped = cv2.warpPerspective(image, M, combined_size, flags = cv2.INTER_LINEAR)
    binary_warped = binary_warped.astype(np.uint8)
    
    # Return warped images
    return binary_warped, warped


## Find midpoint and peaks of left and right line

In [7]:
def get_values(binary_warped):
    # Calculate histogram along x-axis
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    # Find midpoint
    midpoint = np.int(histogram.shape[0]/2)
    # Find left lane line base
    leftx_base = np.argmax(histogram[:midpoint])
    # Find right lane line base    
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Return midpoint, left base and right base
    return midpoint, leftx_base, rightx_base

## Find polynimial fit of left lane and right lane

In [8]:
def get_fits(binary_warped, caller = False):
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint, leftx_base, rightx_base = get_values(binary_warped)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(out_img.shape[0]/nwindows)
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = int(out_img.shape[0]) - (window+1)*window_height
        win_y_high = int(out_img.shape[0]) - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Assingn red color to index belonging to left lane line
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    # Assingn blue color to index belonging to right lane line    
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Return left_fit, right_fit and image 'out_img' if call is from test images pipeline
    # else return only left_fit, right_fit, if call is from pipeline for videos
    if caller:
        return left_fit, right_fit, out_img
    else:
        return left_fit, right_fit


## Calculate Radius of Curvature

In [9]:
def get_radius_of_curvature(ploty, left_fit, right_fit):
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Calculate x co-ordinates belonging to left lane line
    leftx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    # Calculate x co-ordinates belonging to right lane line    
    rightx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]
    # Find max y co-ordinate, point at which we want to find radius of curvature 
    y_eval = np.max(ploty)

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, rightx * xm_per_pix, 2)

    # Calculate the new radii of curvature
    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[
                     1])**2)**1.5) / np.absolute(2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[
                      1])**2)**1.5) / np.absolute(2 * right_fit_cr[0])
    
    # Return radius of curvature
    return left_curverad, right_curverad

## Calculate Offset from center

In [10]:
def get_offset_from_center(binary_warped, left_fit, right_fit):
    # Get y, y co-ordinate at which we want to find the position of vehicle
    y = binary_warped.shape[0]
    # Calculate x co-ordinate for left lane line corresponding to y 
    left_point = left_fit[0] * y**2 + left_fit[1] * y + left_fit[2]
    # Calculate x co-ordinate for right lane line corresponding to y
    right_point = right_fit[0] * y**2 + right_fit[1] * y + right_fit[2]
    # # Define conversions in x from pixels space to meters
    xm_per_pix = 3.7 / 700  # meteres per pixel in x dimension
    # Get center point
    road_midpoint = binary_warped.shape[1] / 2
    # Get center point of car
    car_midpoint = int((right_point + left_point) / 2)
    # Calculate offset by taking difference
    pixels_off_center = road_midpoint - car_midpoint
    # Converting offset from pixel to meter using xm_per_pix
    offset_from_center = xm_per_pix * pixels_off_center
    # Return offset_from_center
    return offset_from_center

## Get Unwarped image using Inverse Perspective

In [11]:
def get_warped_back(dest, left_fitx, right_fitx, ploty):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(dest[:,:,0]).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 255))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (dest.shape[1], dest.shape[0])) 
    # Combine the result with the original image
    warped_back = cv2.addWeighted(dest, 1, newwarp, 0.3, 0)
    
    return warped_back

## Get Final Result

In [12]:
def get_final_result(result, avg_radius, offset_from_center):
    
    # Print radius of curvature on video
    cv2.putText(result, 'Radius of Curvature = {}(m)'.format(avg_radius), (150, 200),
                fontFace=16, fontScale=2, color=(255, 255, 0), thickness=2)
    
    # Check the position of vehicle, Left or Right
    if offset_from_center < 0:
        # Print distance from center on video
        cv2.putText(result, 'Vehicle is {:.2f}m left of center'.format(abs(offset_from_center)), (150, 100),
                        fontFace=16, fontScale=2, color=(255, 255, 0), thickness=2)
    else:
        # Print distance from center on video
        cv2.putText(result, 'Vehicle is {:.2f}m right of center'.format(offset_from_center), (150, 100),
                        fontFace=16, fontScale=2, color=(255, 255, 0), thickness=2)
    # Return result
    return result


## Pipeline to test on test images

In [13]:
# Define counter to save test images in respective folders
i = 0
raw_images = glob.glob('test_images/*.jpg')
# Looping test images through pipeline
for fname in raw_images:
    # Read images
    image = mpimg.imread(fname)
    
    #binary_warped, warped, dest, combined = get_better_warped(image, caller = True)
    binary_warped, warped, dest, combined = get_warped_images(image, caller = True)    
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    left_fit, right_fit, out_img = get_fits(binary_warped, caller = True)
    
    # Get y co-ordinates using np.linspace
    ploty = np.linspace(0, int(binary_warped.shape[0])-1, int(binary_warped.shape[0]))
    # Calculate x co-ordinates belonging to left lane line
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    # Calculate x co-ordinates belonging to right lane line
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Get Radius of Curvature
    left_curverad, right_curverad = get_radius_of_curvature(ploty, left_fit, right_fit)
    
    # Get Unwarped image 'warped_back'
    warped_back = get_warped_back(dest, left_fitx, right_fitx, ploty)
    
    # Get position of vehicle w.r.t center
    offset_from_center = get_offset_from_center(binary_warped, left_fit, right_fit)
    
    # Get final result having radius of curvature and offset
    result = get_final_result(warped_back, int((left_curverad + right_curverad) / 2), offset_from_center)
    
    # Definig list of folders
    folder = ['straight_lines1', 'straight_lines2', 'test1', 'test2', 'test3', 'test4', 'test5', 'test6','test7','test8']
    
    # Create figue to save undistorted image
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    f.tight_layout()
    
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=40)

    ax2.imshow(dest)
    ax2.set_title('Undistorted Image', fontsize=40)
    path = 'output_images/'+folder[i]+'/Undistort.jpg'
    plt.savefig(path)

    # Create figure to save Warped image
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    f.tight_layout()
    
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=40)

    ax2.imshow(warped)
    ax2.set_title('Warped Image', fontsize=40)
    path = 'output_images/'+folder[i]+'/Warped.jpg'
    plt.savefig(path)
    
    # Save binary warped image
    combined = combined.astype(np.uint8)*255
    path = 'output_images/'+folder[i]+'/combined.jpg'
    cv2.imwrite(path, combined)
    
    # Create figure to save color_fit_line.jpg
    f, ax1 = plt.subplots(1,1)
    f.tight_layout()
    path = 'output_images/'+folder[i]+'/color_fit_line.jpg'
    ax1.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.savefig(path)
    
    # Saving output image
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    path = 'output_images/'+folder[i]+'/output.jpg'
    cv2.imwrite(path, result)
    
    # Incrementing counter
    i += 1


## Define Line class

In [14]:
class Line():

    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False
        # x values of the last n fits of the line
        self.recent_xfitted = []
        # average x values of the fitted line over the last n iterations
        self.bestx = None
        # polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        # polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]
        # radius of curvature of the line in some units
        self.radius_of_curvature = None
        # distance in meters of vehicle center from the line
        self.line_base_pos = None
        # difference in fit coefficients between last and new fits
        self.diffs = np.array([0, 0, 0], dtype='float')
        # x values for detected line pixels
        self.allx = None
        # y values for detected line pixels
        self.ally = None

## Create objects Left and Right for left and right line repectively

In [15]:
# Create object for left lane line
Left = Line()
# Create object for right lane line
Right = Line()

## Find new lines using last lines

In [16]:
def get_new_fits(binary_warped, prev_left_fit, prev_right_fit):
    # Define window margin
    margin = 100
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Do not do a blind search but use the previous fits to locate the pixels
    left_lane_inds = ((nonzerox > (prev_left_fit[0] * (nonzeroy**2) + prev_left_fit[1] * nonzeroy + prev_left_fit[2] - margin)) & (
            nonzerox < (prev_left_fit[0] * (nonzeroy**2) + prev_left_fit[1] * nonzeroy + prev_left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (prev_right_fit[0] * (nonzeroy**2) + prev_right_fit[1] * nonzeroy + prev_right_fit[2] - margin)) & (
            nonzerox < (prev_right_fit[0] * (nonzeroy**2) + prev_right_fit[1] * nonzeroy + prev_right_fit[2] + margin)))

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Return None if line is not found 
    if (len(leftx) == 0) or (len(rightx) == 0):
        return None, None

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Return left_fit and right_fit
    return left_fit, right_fit

## Pipeline to process frames of video 

In [17]:
def process_frames(img):
    
    # Define averaging factor alpha
    alpha = 0.8

    # Get warped and undistorted image
    binary_warped, dest = get_warped_images(img)

    # Fit left and right lines
    if (Left.detected == False) or (Right.detected == False):
        Left.current_fit, Right.current_fit = get_fits(binary_warped)
    else:
        Left.current_fit, Right.current_fit = get_new_fits(binary_warped, Left.best_fit, Right.best_fit)

    # Get y co-ordinates using np.linspace()
    yvals = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])

    # Find current_fit and best_fit for left lane line
    if Left.current_fit != None:
        # Define Left.best_fit, if it's None
        if Left.best_fit == None:
            Left.best_fit = Left.current_fit
        
        # Find x co-ordinates belonging to current fit using Left.current_fit and Left.best_fit respectively 
        current_leftx = Left.current_fit[0] * yvals**2 + Left.current_fit[1] * yvals + Left.current_fit[2]
        last_leftx = Left.best_fit[0] * yvals**2 + Left.best_fit[1] * yvals + Left.best_fit[2]
            
        # Calculate new x using x co-ordinates calculated above
        leftx = (alpha*last_leftx) + (1 - alpha)*current_leftx
        
        # Get new co-efficients corresponding to x co-ordinated calculated n last line
        Left.best_fit = np.polyfit(yvals, leftx, 2)
        
        # Set flag Left.detected to True
        Left.detected = True
    else:
        # Set flag Left.detected to False, if Left lane line not found
        Left.detected = False
        # Calculte current x co-ordinates using Left.best_fit
        current_leftx = Left.best_fit[0] * yvals**2 + Left.best_fit[1] * yvals + Left.best_fit[2]

    # Find current_fit and best_fit for right lane line
    if Right.current_fit != None:
        # Define Right.best_fit, if it's None
        if Right.best_fit == None:
            Right.best_fit = Right.current_fit
            
        # Find x co-ordinates belonging to current fit using Right.current_fit and Right.best_fit respectively 
        current_rightx = Right.current_fit[0] * yvals**2 + Right.current_fit[1] * yvals + Right.current_fit[2]
        last_rightx = Right.best_fit[0] * yvals**2 + Right.best_fit[1] * yvals + Right.best_fit[2]
            
        # Calculate new x using x co-ordinates calculated above
        rightx = (alpha*last_rightx) + (1 - alpha)*current_rightx
        
        # Get new co-efficients corresponding to x co-ordinated calculated n last line
        Right.best_fit = np.polyfit(yvals, rightx, 2)
        
        # Set flag Left.detected to True
        Right.detected = True
    else:
        # Set flag Left.detected to False, if Right lane line not found
        Right.detected = False
        # Calculte current x co-ordinates using Right.best_fit
        current_rightx = Right.best_fit[0] * yvals**2 + Right.best_fit[1] * yvals + Right.best_fit[2]
    
    # Get Radius of curvature
    Left.radius_of_curvature, Right.radius_of_curvature = get_radius_of_curvature(yvals, Left.best_fit, Right.best_fit)
    
    # Get Warped Back Image
    warped_back = get_warped_back(dest, current_leftx, current_rightx, yvals)
    
    # Get position of vehicle w.r.t center 
    offset_from_center = get_offset_from_center(binary_warped, Left.best_fit, Right.best_fit)
    
    # Get output image
    result = get_final_result(warped_back, int((Left.radius_of_curvature + Right.radius_of_curvature) / 2), offset_from_center)
    
    # Return result image
    return result

## Process project video

In [18]:
white_output = 'output_images/test-output.mp4'
clip1 = VideoFileClip("test_video.mp4")
white_clip = clip1.fl_image(process_frames) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video output_images/test-output.mp4
[MoviePy] Writing video output_images/test-output.mp4


 97%|███████████████████████████████████████████████████████████████████████████████▉  | 38/39 [00:36<00:00,  1.15it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/test-output.mp4 

Wall time: 55.9 s


In [19]:
white_output = 'output_images/P1-challenge-output.mp4'
clip1 = VideoFileClip("P1-challenge.mp4")
white_clip = clip1.fl_image(process_frames) #NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video output_images/P1-challenge-output.mp4
[MoviePy] Writing video output_images/P1-challenge-output.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [03:39<00:00,  1.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/P1-challenge-output.mp4 



In [20]:
white_output = 'output_images/project-output.mp4'
clip2 = VideoFileClip("project_video.mp4")
white_clip = clip2.fl_image(process_frames) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video output_images/project-output.mp4
[MoviePy] Writing video output_images/project-output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [17:09<00:00,  1.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project-output.mp4 

Wall time: 17min 17s



## Functions defined to experiment with detection of lane line

In [33]:
path = 'test_images/test1.jpg'
img = cv2.imread(path)
hls = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
lower_yellow = np.array([20,100,100])
upper_yellow = np.array([30,255,255])
mask1 = cv2.inRange(hls, lower_yellow, upper_yellow)

mask0 = cv2.inRange(hls, np.array([0,0,240]), np.array([255,15,255]))

combined1 = mask1+mask0

mask = np.zeros_like(combined1)
ignore_mask_color = 255
vertices = np.array([[(500, 450),(800, 450), (1100, 700), (100,700)]], dtype=np.int32)
cv2.fillPoly(mask, vertices, ignore_mask_color)
combined1 = cv2.bitwise_and(combined1, mask)

cv2.imwrite('testing.jpg',combined1)

True

In [34]:
path = 'test_images/test5.jpg'
img = cv2.imread(path)
binary_warped, warped, dest, combined = get_better_warped(img, caller = True)
cv2.imwrite('testing1.jpg',combined)

True

In [29]:
def get_better_warped(image, caller = False):
    
    dest = cv2.undistort(image, mtx, dist, None, mtx)
    img = np.copy(dest)
    
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    lower_yellow = np.array([20,100,100])
    upper_yellow = np.array([30,255,255])
    mask1 = cv2.inRange(hls, lower_yellow, upper_yellow)

    mask0 = cv2.inRange(hls, np.array([255,255,255]), np.array([255,255,255]))

    combined1 = mask1+mask0

    mask = np.zeros_like(combined1)
    ignore_mask_color = 255
    vertices = np.array([[(500, 450),(800, 450), (1100, 700), (100,700)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    combined = cv2.bitwise_and(mask, combined1)
    combined = combined.astype(np.uint8)
    final = np.zeros_like(combined)
    final[(combined == 255)] = 1
    #combined = combined // 255
    combined = final
    combined_size = combined.shape[::-1]
    
    binary_warped = cv2.warpPerspective(combined, M, combined_size, flags = cv2.INTER_LINEAR)
    warped = cv2.warpPerspective(image, M, combined_size, flags = cv2.INTER_LINEAR)
    binary_warped = binary_warped.astype(np.uint8)
    
    if caller:
        return binary_warped, warped, dest, combined
    else:
        return binary_warped, dest

In [30]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    # Define a blank mask to start with
    mask = np.zeros_like(img)   
    
    # Define a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 255, 255], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    xtl, xtr, xbl, xbr = [], [], [], []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if abs(y1 - y2) > 20 and x1 != x2:
                slope = (y2 - y1) / (x2 - x1)
                b = y2 - (slope * x2)
                y = img.shape[0]
                x = int((y - b) / slope)
                x_ = int((325 - b) / slope)
                if slope < 0:
                    xbl.append(x)
                    xtl.append(x_)
                else:
                    xbr.append(x)
                    xtr.append(x_)
    xtlmean = int(np.mean(xtl))
    xblmean = int(np.mean(xbl))
    xtrmean = int(np.mean(xtr))
    xbrmean = int(np.mean(xbr))
    yb = img.shape[0]
    cv2.line(img, (xtlmean,600), (xblmean,720), color, thickness)
    cv2.line(img, (xtrmean,600), (xbrmean,720), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)